# Topic Model Analysis

Todo: 1. industry : cusip to naics mapping 2. Topic evloution

In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

from compute_topic import *
from manage_path import *
from topic_model_analysis import *

import plotly
plotly.offline.init_notebook_mode(connected=True) 
import plotly.graph_objs as go
import plotly.io as pio
from itertools import repeat

import multiprocessing
from collections import deque

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from collections import OrderedDict

## Topic Evolution
1. Dc_v1: 1 document for buy and sell, per dealer, per day.
2. Dc_v2: 1 document for buy and 1 document for sell, per dealer, per day.
3. Dc_v3: Either DC_v1 or DC_v2 without the 2 (4) documents representing the Source_seller and Source_buyer.
4. Tc_v1: 1 document for (buyer,seller,year)

In [3]:
result_directory = get_result_directory()
get_document_item_vectorize = np.vectorize(get_document_item)
#topic_matrix['dealer'] = pd.Series(list(zip(get_document_item_vectorize(topic.index,0),get_document_item_vectorize(topic.index,1)))).values

### Dc_v1

In [ ]:
Dc_v1 = pd.read_csv(result_directory/'Dc_v1_250topics.csv',index_col=0)

In [ ]:
Dc_v1.head(3)

In [ ]:
Dc_v1['dealer'] = get_document_item_vectorize(Dc_v1.index,0)

In [ ]:
Dc_v1.index = pd.to_datetime(get_document_item_vectorize(Dc_v1.index,1))

In [ ]:
Dc_v1.head(3)

In [ ]:
count_matrix = Dc_v1.groupby(["dealer"]).count().sort_index()

In [ ]:
ax = sns.heatmap(count_matrix,cmap="YlGnBu")

In [ ]:
ax = sns.heatmap(Dc_v1.groupby(by=Dc_v1.index).sum().values.tolist(),cmap="YlGnBu")

In [ ]:
ax = sns.heatmap(Dc_v1.groupby(by=Dc_v1.index).mean().values.tolist(),vmin=0,vmax=1,center=0.5,cmap="YlGnBu")

### Dc_v2

In [ ]:
Dc_v2 = pd.read_csv(result_directory/'Dc_v2_250topics.csv',index_col=0)

In [ ]:
Dc_v2.head(3)

In [ ]:
Dc_v2['dealer'] = pd.Series(list(zip(get_document_item_vectorize(Dc_v2.index,0),get_document_item_vectorize(Dc_v2.index,2)))).values

In [ ]:
Dc_v2.index = pd.to_datetime(get_document_item_vectorize(Dc_v2.index,1))

In [ ]:
Dc_v2.head(3)

In [ ]:
ax = sns.heatmap(Dc_v2.groupby(by=Dc_v2.index).sum().values.tolist(),cmap="YlGnBu")

In [ ]:
ax = sns.heatmap(Dc_v2.groupby(by=Dc_v2.index).mean().values.tolist(),vmin=0,vmax=1,center=0.5,cmap="YlGnBu")

### Dc_v3

In [14]:
Dc_v3 = pd.read_csv(result_directory/'Dc_v3_50topics.csv',index_col=0)

In [15]:
Dc_v3.head(3)

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
"10,2014-01-09,S",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"10,2014-01-14,S",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"10,2014-01-21,S",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
Dc_v3['dealer'] = pd.Series(list(zip(get_document_item_vectorize(Dc_v3.index,0),get_document_item_vectorize(Dc_v3.index,2)))).values
Dc_v3.index = pd.to_datetime(get_document_item_vectorize(Dc_v3.index,1))

In [17]:
Dc_v3.head(3)

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,dealer
2014-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(10, S)"
2014-01-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(10, S)"
2014-01-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(10, S)"


In [8]:
#Dc_v3.groupby(by=Dc_v3.index).sum().describe()

In [11]:
Dc_v3_topicXtime_sum = Dc_v3.groupby(by=Dc_v3.index).sum()

In [12]:
#Dc_v3.groupby(by=Dc_v3.index).sum().to_csv('Dc_v3_50topics_topicXtime_sum.csv')

In [13]:
model_name = "Dc_v3"
heatmap_data = [
    go.Heatmap(
        z=Dc_v3_topicXtime_sum.values.tolist(),
        #zmin=0,
        #zmax=1,
        x=Dc_v3_topicXtime_sum.columns,
        y=Dc_v3_topicXtime_sum.index,
        colorscale='Jet',
    )
]

layout = go.Layout(
    title='Dc_v3_50topics_topicXtime_sum',
    xaxis={'title':'TopicID'},
    yaxis={'title':'Time'},
    width=1400,
    height=1000,
    #zaxis={'title':'Weighting Sum'},
)

fig = go.Figure(data=heatmap_data, layout=layout)
pio.write_image(fig, str("Dc_v3_50topics_topicXtime_sum.png"))

In [14]:
Dc_v3_topicXtime_count = Dc_v3.groupby(by=Dc_v3.index).count().drop('dealer',axis=1)#[Dc_v3.columns[0:2]]

In [15]:
#Dc_v3.groupby(by=Dc_v3.index).count().to_csv('Dc_v3_50topics_topicXtime_count.csv')

In [16]:
model_name = "Dc_v3"
heatmap_data = [
    go.Heatmap(
        z=Dc_v3_topicXtime_count.values.tolist(),
        #zmin=0,
        #zmax=1,
        x=Dc_v3_topicXtime_count.columns,
        y=Dc_v3_topicXtime_count.index,
        colorscale='Jet',
    )
]

layout = go.Layout(
    title='Dc_v3_50topics_topicXtime_count',
    xaxis={'title':'TopicID'},
    yaxis={'title':'Time'},
    #zaxis={'title':'Weighting Sum'},
    width=1400,
    height=1000,
)

fig = go.Figure(data=heatmap_data, layout=layout)
pio.write_image(fig, str("Dc_v3_50topics_topicXtime_count.png"))

### Snakey

In [242]:
THRESHOLD = 0.65
Dc_v3_dealerXtopic_sum = Dc_v3.groupby(by='dealer').mean()
Dc_v3_dealerXtopic_sum.index = Dc_v3_dealerXtopic_sum.index.format()
Dc_v3_dealerXtopic_sum = pd.DataFrame(Dc_v3_dealerXtopic_sum.stack().reset_index().rename({'level_0':'dealer','level_1':'topicID',0:'values'},axis=1))
#Dc_v3_dealerXtopic_sum['topicID'] = Dc_v3_dealerXtopic_sum['topicID'].astype(np.int)
Dc_v3_dealerXtopic_sum = Dc_v3_dealerXtopic_sum[Dc_v3_dealerXtopic_sum['values']>=THRESHOLD].copy()
Dc_v3_dealerXtopic_sum['B/S'] = Dc_v3_dealerXtopic_sum.apply(lambda x: str(x['dealer'])[-2] ,axis=1)
Dc_v3_dealerXtopic_sum['dealer'] = Dc_v3_dealerXtopic_sum.apply(lambda x: str(x['dealer']).split(',')[0][1:] ,axis=1)
"""
# 
Dc_v3_dealerXtopic_sum = Dc_v3_dealerXtopic_sum[(Dc_v3_dealerXtopic_sum['topicID']=='0') | (Dc_v3_dealerXtopic_sum['topicID']=='30')|
                                                (Dc_v3_dealerXtopic_sum['topicID']=='15') | (Dc_v3_dealerXtopic_sum['topicID']=='46')|
                                               (Dc_v3_dealerXtopic_sum['topicID']=='48') | (Dc_v3_dealerXtopic_sum['topicID']=='35')|
                                               (Dc_v3_dealerXtopic_sum['topicID']=='41') | (Dc_v3_dealerXtopic_sum['topicID']=='43')].copy()
"""
"""
# Topics on top
Dc_v3_dealerXtopic_sum = Dc_v3_dealerXtopic_sum[(Dc_v3_dealerXtopic_sum['topicID']=='36') | (Dc_v3_dealerXtopic_sum['topicID']=='21')|
                                                (Dc_v3_dealerXtopic_sum['topicID']=='13') | (Dc_v3_dealerXtopic_sum['topicID']=='28')|
                                               (Dc_v3_dealerXtopic_sum['topicID']=='12') | (Dc_v3_dealerXtopic_sum['topicID']=='39')|
                                               (Dc_v3_dealerXtopic_sum['topicID']=='1') | (Dc_v3_dealerXtopic_sum['topicID']=='18')].copy()
"""
"""
# Topics in middle (wide dealers)
Dc_v3_dealerXtopic_sum = Dc_v3_dealerXtopic_sum[(Dc_v3_dealerXtopic_sum['topicID']=='14') | (Dc_v3_dealerXtopic_sum['topicID']=='15')|
                                                (Dc_v3_dealerXtopic_sum['topicID']=='47') | (Dc_v3_dealerXtopic_sum['topicID']=='26')|
                                               (Dc_v3_dealerXtopic_sum['topicID']=='8') | (Dc_v3_dealerXtopic_sum['topicID']=='38')|
                                               (Dc_v3_dealerXtopic_sum['topicID']=='34') | (Dc_v3_dealerXtopic_sum['topicID']=='46')|
                                               (Dc_v3_dealerXtopic_sum['topicID']=='49') | (Dc_v3_dealerXtopic_sum['topicID']=='22')].copy()
"""
"""
# Buy topics
Dc_v3_dealerXtopic_sum = Dc_v3_dealerXtopic_sum[(Dc_v3_dealerXtopic_sum['topicID']=='1') | (Dc_v3_dealerXtopic_sum['topicID']=='16')|
                                                (Dc_v3_dealerXtopic_sum['topicID']=='33') | (Dc_v3_dealerXtopic_sum['topicID']=='17')|
                                               (Dc_v3_dealerXtopic_sum['topicID']=='7') | (Dc_v3_dealerXtopic_sum['topicID']=='22')|
                                               (Dc_v3_dealerXtopic_sum['topicID']=='27') | (Dc_v3_dealerXtopic_sum['topicID']=='2')|
                                               (Dc_v3_dealerXtopic_sum['topicID']=='9')].copy()
"""
# Sell topics
Dc_v3_dealerXtopic_sum = Dc_v3_dealerXtopic_sum[(Dc_v3_dealerXtopic_sum['topicID']=='0') | (Dc_v3_dealerXtopic_sum['topicID']=='45')|
                                                (Dc_v3_dealerXtopic_sum['topicID']=='15') | (Dc_v3_dealerXtopic_sum['topicID']=='46')|
                                               (Dc_v3_dealerXtopic_sum['topicID']=='35') | (Dc_v3_dealerXtopic_sum['topicID']=='41')].copy()
Dc_v3_dealerXtopic_sum = Dc_v3_dealerXtopic_sum.sort_values(by=['topicID','dealer'])

In [243]:
Dc_v3_dealerXtopic_sum.head()

,dealer,topicID,values,B/S
136,1023,0,0.676330,S
265,1040,0,0.698810,S
278,1042,0,0.750312,B
307,1042,0,0.718283,S
802,1084,0,0.673333,S


In [244]:
from sklearn import preprocessing
dealer_le = preprocessing.LabelEncoder()
dealer_le.fit(Dc_v3_dealerXtopic_sum['dealer'])
dealer_transform = dealer_le.transform(Dc_v3_dealerXtopic_sum['dealer'])
dealer_inverse_transform = dealer_le.inverse_transform(dealer_transform)

topic_le = preprocessing.LabelEncoder()
topic_le.fit(Dc_v3_dealerXtopic_sum['topicID'])
topic_transform = topic_le.transform(Dc_v3_dealerXtopic_sum['topicID'])
topic_inverse_transform = topic_le.inverse_transform(topic_transform)

In [245]:
Dc_v3_dealerXtopic_sum['topicID'].nunique()

6

In [246]:
# Adjust dealer_label_position
Dc_v3_dealerXtopic_sum['dealer_encoding'] = dealer_transform
topicID_size = Dc_v3_dealerXtopic_sum['topicID'].nunique()
Dc_v3_dealerXtopic_sum['dealer_label_position'] = Dc_v3_dealerXtopic_sum.apply(lambda x: x['dealer_encoding'] + topicID_size, axis=1)
Dc_v3_dealerXtopic_sum['topic_encoding'] = topic_transform
Dc_v3_dealerXtopic_sum['topic_position'] = topic_transform
label = Dc_v3_dealerXtopic_sum['topicID'].values.tolist().append(Dc_v3_dealerXtopic_sum['dealer'].values.tolist())

In [247]:
# Create labels
topic_label = list(Dc_v3_dealerXtopic_sum['topicID'].unique())
dealer_label = list(Dc_v3_dealerXtopic_sum.sort_values(by=['dealer_encoding'])['dealer'].unique())
label = []
label.extend(topic_label)
label.extend(dealer_label)

In [248]:
# Create Label's Colors
label_color = len(topic_label)*['darkred',]
label_color.extend(len(dealer_label)*['black',])

In [249]:
# Create Link Colors based on B/S 
b_count = Dc_v3_dealerXtopic_sum.groupby(['B/S']).count()['dealer'][0]
s_count = Dc_v3_dealerXtopic_sum.groupby(['B/S']).count()['dealer'][1]
link_color = b_count*['deepskyblue',]
link_color.extend(s_count*['orange',])
# Sort by B/S to match link color arrays 
Dc_v3_dealerXtopic_sum.sort_values(by=['B/S'],inplace=True)

In [250]:
Dc_v3_dealerXtopic_sum.to_csv(str("Dc_v3_50topics_THRESHOLD={}").format(THRESHOLD))

In [251]:
data = dict(
    type='sankey',
    orientation = "h",
    valueformat = ".4f",
    node = dict(
      pad = 100,
      thickness = 30,
      line = dict(
        color = "black",
        width = 0.5
      ),
      label = label,
      color = label_color
    ),
    link = dict(
      source = Dc_v3_dealerXtopic_sum['dealer_label_position'],
      target = Dc_v3_dealerXtopic_sum['topic_position'],
      value = Dc_v3_dealerXtopic_sum['values'],
      #label = inverse_transform
      color = link_color
  ))

title = str("Dc_v3_50topics_THRESHOLD={}").format(THRESHOLD)
layout =  dict(
    title = title,
    font = dict(
      size = 20
    ),
    width=1500,
    height=5000,
)

fig = dict(data=[data], layout=layout)
#plotly.offline.iplot(fig, validate=False)
pio.write_image(fig, "{}.png".format(title))
plotly.offline.plot(fig, filename =  "{}.html".format(title), auto_open=False)

'file://C:\\Users\\raymo\\UMD\\Research\\FINRA_TRACE\\TopicModeling\\Dc_v3_50topics_THRESHOLD=0.65.html'

In [ ]:
Dc_v3_dealerXtopic_sum['topicID'].values

### Tc_v1

In [ ]:
Tc_v1 = pd.read_csv(result_directory/'Tc_v1_250topics.csv',index_col=0)

In [ ]:
Tc_v1.head(3)

In [ ]:
Tc_v1['dealer'] = pd.Series(list(zip(get_document_item_vectorize(Tc_v1.index,0),get_document_item_vectorize(Tc_v1.index,1)))).values

In [ ]:
Tc_v1.index = pd.to_datetime(get_document_item_vectorize(Tc_v1.index,2))

### topicXtime heatmap

#### Try Plotly

In [ ]:
#ax = sns.heatmap(topic_matrix.values.tolist(),vmin=0,vmax=1,center=0.5)

In [ ]:
count_matrix = topic_matrix.groupby(["dealer"]).count()

In [ ]:
summary = topic_matrix.describe().T

In [ ]:
summary['mean'].plot()

In [ ]:
def get_dealer_by_ID(matrix,dealer_id,matrix_name):
    result = matrix.loc[matrix['dealer'] == dealer_id].copy().drop(labels='dealer',axis=1)
    return (result,dealer_id,matrix_name)

In [ ]:
dealer_df_list = list(map(lambda x: get_dealer_by_ID(topic_matrix,x,"Dc_v1"),list(topic_matrix['dealer'].unique())))
len(dealer_df_list)

In [ ]:
#deque(map(topicXtime,dealer_df_list))

In [ ]:
pio.orca.config
plotly.io.orca.status

cpu_cores = multiprocessing.cpu_count() - 1
pool = multiprocessing.Pool(cpu_cores)
pool.map(topicXtime_plotly_parallel,dealer_df_list)
pool.close()

#### Try mapplotlib

In [ ]:
cpu_cores = int(multiprocessing.cpu_count()/2) -1
pool = multiprocessing.Pool(cpu_cores)

In [ ]:
pool.starmap(topicXtime_matplotlib,dealer_df_list)
pool.close()

#### Try Seaborn

In [ ]:
sns.set()

In [ ]:
month_list = list(OrderedDict.fromkeys(dealer_df_list[0][0].index.strftime('%Y-%m')))

In [ ]:
ax = sns.heatmap(dealer_df_list[0][0].T.values,vmin=0,vmax=1,center=0.5,xticklabels=[0,50,100,150,200,250],yticklabels=month_list)

num_x_ticks = 5
# the index of the position of yticks
xticks = np.linspace(0, len(dealer_df_list[0][0].columns) - 1, num_x_ticks, dtype=np.int)

num_y_ticks = len(month_list)
# the index of the position of yticks
yticks = np.linspace(0, len(dealer_df_list[0][0].index) - 1, num_y_ticks, dtype=np.int)

#ax.set_xticks(xticks)
#ax.set_yticks(yticks)
ax.set_autoscaley_on(True)
ax.set_title('test')

In [ ]:
import matplotlib.dates as mdates
ax = sns.heatmap(dealer_df_list[0][0].T,vmin=0,vmax=1,center=0.5)

ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
#ax.xaxis.set_minor_locator(mdates.DayLocator())

#ax.set_xlim(dealer_df_list[0][0].index[0], dealer_df_list[0][0].index[-1])
#ax.format_xdata = mdates.DateFormatter('%Y-%m-%d')

figure = ax.get_figure()
figure.savefig('test.png', dpi=400)

In [ ]:
ax = sns.heatmap(dealer_df_list[0][0].T,vmin=0,vmax=1,center=0.5)

In [ ]:
dealer_df_list[0][0].index[-1]

In [ ]:
yticks

In [ ]:
xticks

In [ ]:
figure = ax.get_figure()
figure.savefig('test.png', dpi=400)
#figure.savefig('test.svg', format="svg")

In [ ]:
ax.fig.autofmt_ydate()

In [ ]:
topic_matrix.corr().head()

## Topic Terms Distribution
Below is the topic terms ditrutbution demo. Since we already got Document X Topics, we want to have a Topic X Terms too.

In [ ]:
num_topics = 50
model = load_model('Dc_v3',num_topics)

In [ ]:
topic_terms_distribution = pd.DataFrame(model.get_topics())

In [ ]:
topic_terms_distribution.head()

## Industy Analysis

In [ ]:
data_path = get_pickle_directory() / 'TRACE2014_jinming.pkl'
data = pd.read_pickle(data_path)

In [ ]:
data.columns

In [ ]:
print("We have {} rows of data".format(data.shape[0]))

In [ ]:
def fix_NAICS_Code(NAICS_Code):
    "Fix the problem that FISD omits 0 if NAICS_Code ends with 0"
    if len(str(NAICS_Code))==5:
        NAICS_Code = NAICS_Code+'0'
    return NAICS_Code
fix_NAICS_Code_vectorize = np.vectorize(fix_NAICS_Code)
data['NAICS_CODE'] = fix_NAICS_Code_vectorize(data['NAICS_CODE'].values)

In [ ]:
data = data[['BOND_SYM_ID', 'INDUSTRY_GROUP', 'INDUSTRY_CODE', 'PARENT_ID', 'NAICS_CODE','SIC_CODE']].copy()

In [ ]:
data['INDUSTRY_CODE'].value_counts().plot.pie()

In [ ]:
# Get the NAICS_code lookup table
NAICS_code_path = get_dataset_directory() / 'NAICS_Code_Lookup.csv'
NAICS_code = pd.read_csv(NAICS_code_path,dtype={'Code':str,'Description':str})

In [ ]:
# Group by BOND_SYM_ID and NAICS_CODE of 
NAICS_CODE_count = data['NAICS_CODE'].value_counts()
# Series to DataFrame
NAICS_CODE_count = NAICS_CODE_count.to_frame(name='count')

In [ ]:
NAICS_code_total = NAICS_code.merge(NAICS_CODE_count,left_on='Code',right_index=True).sort_values(by="count",ascending=False)
NAICS_code_total['percentage'] = NAICS_code_total['count']/data.shape[0]

In [ ]:
NAICS_code_total.head()

In [ ]:
(NAICS_code.merge(NAICS_CODE_count,left_on='Code',right_index=True).sort_values(by="count",ascending=False)['count']/data.shape[0]).sum()

In [ ]:
data.groupby(by=['NAICS_CODE'])['BOND_SYM_ID'].nunique().sort_values(ascending=False).head()

In [ ]:
data.groupby(by=['BOND_SYM_ID'])

In [ ]:
Bond_X_Industry = data.drop_duplicates(['BOND_SYM_ID'])

In [ ]:
Bond_X_Industry.sort_values(by=['BOND_SYM_ID']).head()

## Individual Check

## Convergence Test

In [ ]:
import re
import matplotlib.pyplot as plt
def convergence_likelyhood(model_name,num_topics):
    p = re.compile("(-*\d+\.\d+) per-word .* (\d+\.\d+) perplexity")
    matches = [p.findall(l) for l in open('../LDAModel/{}_{}topics.log.txt'.format(model_name,num_topics))]
    matches = [m for m in matches if len(m) > 0]
    tuples = [t[0] for t in matches]
    perplexity = [float(t[1]) for t in tuples]
    liklihood = [float(t[0]) for t in tuples]
    iter = list(range(0,len(tuples)*10,10))
    plt.plot(iter,liklihood,c="black")
    plt.ylabel("log liklihood")
    plt.xlabel("iteration")
    plt.title("Topic Model Convergence")
    plt.grid()
    plt.savefig("{}_{}topics.pdf".format(model_name,num_topics))
    plt.close()

In [ ]:
#convergence_likelyhood("matrix_1",250)